# Charles Mendelson Pinata Farm Analytics Project

### Design decisions:
#### 1. Rather than load all of the .csv files into one data frame to manipulate, I kept them separate for 2 reasons, the first was I ran into memory issues when I tried combining them. The second was if something failed I would have staged backups that would speed up development.
#### 2. I chose to run sentiment analysis only on English language tweets. While I could have analyzed other languages I would have no ability to QA the findings
#### 3. I have never done sentiment analysis before, and the method I chose was one that scored on multiple emotions rather than positive negative axis.
#### 4. The emotional lexicon came from http://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm
#### 5. To speed up processing, I removed neutral words from the lexicon
#### 6. The goal is to prepare the data for analysis in tableau
#### 7. I have never done sentiment analysis before, and this solution felt a little brute forcey, but I did think of another of optimizations that could speed it up.
#### 8. If I had more time, I'd refactor the code to be more concise, but it does work.
#### 9. The sentiment analysis is crude. It produces counts of words with more sentiment, but it doesn't handle sarcasm, negatives, or other emotional nuance well.

### Modules Needed

In [ ]:
import pandas as pd
import numpy as np
import glob
from nltk import word_tokenize
from sqlalchemy import create_engine
from nltk.stem.snowball import SnowballStemmer
import re
from datetime import datetime
from nltk.corpus import stopwords
import psycopg2

functions needed to clean data

In [ ]:
#creates mask for loc
def mask(df_w_column, value):
    return df_w_column == value

#function to filter based on mask, currently only handles one filter, will be extended to handle more
def mask_filter(df, masks, conditions=1, invert=False, condition_type = None):
    if conditions == 1 & invert == False:
        return df.loc[masks].reset_index(drop=True)
    if conditions == 1 & invert == True:
       return df.loc[~masks].reset_index(drop=True)

globs tweet .csv data and cleans each csv for sentiment analysis

In [ ]:
path = r'take_home_assignment/tweets'
test_path = r'take_home_assignment/test_path'
def batch_csvs(path):

    all_files = glob.glob(path + "/*.csv")


    destination = r'take_home_assignment/cleaned_tweets/'
    # an NLTK list of words that are fill words
    stopword_pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    # The other regexes are to handle common data cleanup problems
    url_pattern = re.compile(pattern=r'http\S+', flags=re.MULTILINE)
    at_pattern = re.compile(pattern=r'@\S+', flags=re.MULTILINE)
    hash_pattern = re.compile(pattern=r'#.*?(?=\s|$)', flags=re.MULTILINE)
    special_char_pattern = re.compile(pattern=r'[^a-zA-Z0-9 -]', flags=re.MULTILINE)
    counter = 0
    for filename in all_files:
        print(filename)
        df = pd.read_csv(filename, index_col=None, header=0)
        mask_lang = mask(df_w_column=df['lang'], value='en')
        df = df.loc[mask_lang].reset_index(drop=True)
        # Ideally I would make these more concise, possibly with a list comprehension using pd.apply
        df['cleaned_text'] = df['text'].str.lower()
        df['cleaned_text'] = df['cleaned_text'].replace(to_replace=url_pattern, value='', regex=True)
        df['cleaned_text'] = df['cleaned_text'].replace(to_replace=at_pattern, value='', regex=True)
        df['hashtags'] = df['cleaned_text'].str.findall(r'#.*?(?=\s|$)')
        df['cleaned_text'] = df['cleaned_text'].replace(to_replace=hash_pattern, value='', regex=True)
        df['cleaned_text'] = df['cleaned_text'].replace(to_replace=special_char_pattern, value='', regex=True)
        # removes
        df['cleaned_text'] =  df['cleaned_text'].replace(to_replace=stopword_pattern, value='', regex=True)
        df['length'] = df['cleaned_text'].str.len()
        # Removes rows where the tweet had no characters left after the regex operations
        length_mask = df['length'] > 0
        df = df.loc[length_mask]
        df.to_csv(f'{destination}{counter}_english_lang_cleaned_tweets.csv', sep='\t'
                  , encoding='utf8')
        counter +=1



batch_csvs(path=test_path)

operation to score tweets (this got a little messy because I needed to move it out of the function so it would be cached in memory

In [ ]:
# cleans up the corpus

lexicon_file_path = 'NRC-Emotion-Lexicon/' \
                    'NRC-Emotion-Lexicon-v0.92/' \
                    'NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'

#grabs the lexicon
df_emolex = pd.read_csv(lexicon_file_path,
                        names=["word", "emotion", "association"],
                        sep='\t')
#pivots the lexicon so it is in a usable format
df_emolex_words_outer = df_emolex.pivot(index='word',
                               columns='emotion',
                               values='association').reset_index()

#sums the emotional value of each word in the lexicon
df_emolex_words_outer['total'] = df_emolex_words_outer[['anger', 'anticipation', 'disgust'
, 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust']].sum(axis=1)

# creates the dataframe used in the function to score the words
emotions = df_emolex_words_outer.columns.drop('word')
# creates a mask to remove neutral words from the lexicon
mask_neutral_words = mask(df_emolex_words_outer['total'], 0)
# filters the lexicon using the above mask
df_emolex_words_outer = mask_filter(df_emolex_words_outer, mask_neutral_words, invert=True)
# gets the first char from each word
df_emolex_words_outer['first_char'] = df_emolex_words_outer['word'].str[0]
df_emolex_words_outer.columns = df_emolex_words_outer.columns.get_level_values('emotion')
# sets the word to be the index to use an iloc function instead of a loc function
df_emolex_words_outer =  df_emolex_words_outer.set_index(['word'])
emotions


This is the meat of the program, and it got very messy through a series of refactors to optimize
I was able to improve the performance of the algorithm by about 50%, but it's probably putting lipstick on a pig.
I have several thoughts on how to wring more performance out of this but I don't have any more time

In [ ]:
def emotional_scoring(df, column, lexicon, emotions_list):
    new_df = df.copy()
    df_emolex_words = lexicon
    emotions = emotions_list

    # lexicon_file_path = 'NRC-Emotion-Lexicon/' \
    #                 'NRC-Emotion-Lexicon-v0.92/' \
    #                 'NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'
    #
    # df_emolex = pd.read_csv(lexicon_file_path,
    #                         names=["word", "emotion", "association"],
    #                         sep='\t')
    #
    # df_emolex_words = df_emolex.pivot(index='word',
    #                                columns='emotion',
    #                                values='association').reset_index()
    #
    # df_emolex_words['total'] = df_emolex_words[['anger', 'anticipation', 'disgust'
    # , 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust']].sum(axis=1)
    #
    # mask_neutral_words = mask(df_emolex_words['total'], 0)
    # df_emolex_words = mask_filter(df_emolex_words, mask_neutral_words, invert=True)
    # df_emolex_words['first_char'] = df_emolex_words['word'].str[0]
    # df_emolex_words = df_emolex_words.iloc[:,0:-2]


    # emotions = df_emolex_words.columns.drop('word')

    emo_df = pd.DataFrame(0, index=df.index, columns=emotions)


    #to stem words in the text to better match them
    stemmer = SnowballStemmer('english')

    start = datetime.now()
    for i, row in new_df.iterrows():
        # just to track progress as it goes to help me time and benchmark performance
        if i % 10000 == 0:
            tenthousand = datetime.now()
            delta = tenthousand - start
            print(f'it took {delta} to process {i} rows')
        else:
            document = word_tokenize(new_df.loc[i][column])
            for word in document:
                word = stemmer.stem(word.lower())
                if len(word)< 2:
                    pass
                elif len(word) == 2 and word != 'ay':
                    pass
                else:
                # subsets the dataframe to only loop through matching first characters
                # I could speed it up further with a second character and do a second filter operation
                    first_char = str(word[0])
                # # print(f'{word} : {first_char}')
                    mask_first_char =df_emolex_words['first_char'] == first_char
                    df_emolex_words_masked = df_emolex_words.loc[mask_first_char]
                    emo_score = df_emolex_words_masked[df_emolex_words_masked.index == word]
                    if not emo_score.empty:
                        for emotion in list(emotions):
                            if emotion != 'first_char':
                        # print(emotion)
                                emo_df.at[i, emotion] += emo_score[emotion]
                            else:
                                pass

    new_df = pd.concat([new_df, emo_df], axis=1)

    return new_df


In [ ]:
df_test = pd.read_csv(r'take_home_assignment/cleaned_tweets/0_english_lang_cleaned_tweets.csv', sep='\t')

start = datetime.now()
print(start)
df_emotionally_scored_test = emotional_scoring(df_test, 'cleaned_text', df_emolex_words_outer, emotions)
end = datetime.now()
print(end)

In [ ]:
df_emotionally_scored_test.to_csv('scored_tweets_test.csv', sep='\t', encoding='utf8')

In [ ]:
df_emotionally_scored_test.head(20)

globs the cleaned files and runs them through the emotional scoring function and saves them in a separate folder


In [ ]:
cleaned_tweets_path = r'take_home_assignment/cleaned_tweets'
def emotional_glob(path):

    destination = r'take_home_assignment/emotionally_scored_tweets/'
    all_files = glob.glob(path + "/*.csv")

    counter = 0
    for filename in all_files:
        df = pd.read_csv(filename, sep = '\t')
        print('start:'+ str(datetime.now()))
        df_output = emotional_scoring(df,'cleaned_text')
        print('end' + str(datetime.now()))
        df_output.to_csv(f'{destination}{counter}_emotional_tweets.csv', sep='\t'
                  , encoding='utf8')
        counter+=1



emotional_glob(cleaned_tweets_path)

